# Obtencion de los datos
## obtencion de las paginas
Para obtener los datos de las enfermedades busque primero una pagina web que tuviera una buena cantidad de paginas con informacion sobre enfermedades y encontre dos https://www.onmeda.es y https://cuidateplus.marca.com primero experimente con los datos que tiene cuidateplus, obtuve primero los link a las paginas de las enfermedades, con el siguiente codigo en Python3

In [ ]:
import time
import os

l=ord("A")
while l<=ord("Z"):
	dir=("https://cuidateplus.marca.com/enfermedades/%1s.html"%(str(unichr(l))))
	os.system( "wget "+dir)
	time.sleep(10)
	l+=1

Con ese codigo obtuve las que contienen la descripcion de las enfermedades en orden alfabetico
Despues con el siguiente codigo obtuve las paginas que llevan directamente a las enfermedades, esto con el siguiente codigo.

In [ ]:
import nltk
import sys  
from urllib.request import urlopen
from bs4 import BeautifulSoup
import os

def gethref(texto):
	buf=""
	cond=False
	salida=[]
	for lineas in texto:
		print(lineas.get("href"))
	return salida


dirBase="./paginasBase/"
l=ord("A")
while l<=ord("Z"):
	url = dirBase+str(chr(l))+".html"
	html_doc = open(url,"r").read()
	soup = BeautifulSoup(html_doc, 'html.parser')
	link = soup.find_all('a')
	# print(link)
	print (gethref(link))
	input("borrar")
	os.system("clear")
	l+=1
	print("------------------------------------------------------Aqui-----------------")
# print(soup.get_text())

Use BeautifulSoup4 para obtener los links y despues manualmente copie los conjuntos de links a otro documento con las 299 enfermedades que tiene este sitio web. Despues con el siguiente codigo obtuve las paginas html con la informacion de las enfermedades:

In [ ]:
import time
import os

with open("paginas.txt","r") as f:
	links=f.readlines()
	for l in links:
		os.system( "wget "+l)
		time.sleep(20) 

Una vez con los html de las enfermedades vamos a obtner los sintomas de cada enfermedad y los guardaremos en un csv, con el siguiente codigo:

In [ ]:
import nltk
import sys  
from urllib.request import urlopen
from bs4 import BeautifulSoup
import os
from utilidades import *
import csv

csvfile=open("datos.csv","w",newline='')
dataWriter=csv.writer(csvfile, delimiter=' ',quotechar='|', quoting=csv.QUOTE_MINIMAL)

listaDeSecciones=["Síntomas"] #la informacion que deseamos sacar
# dirBase="./paginasEspecificas/"
dirBase=""
for nombre in open(dirBase+"htmls.txt","r").read().splitlines():
	# input("hola")
	# print(nombre)
	fhtml=open(dirBase+nombre).read()
	soup = BeautifulSoup(fhtml,'html.parser')
	secciones = soup.find_all('h2',attrs={'class':'c'})
	ant=0
	act=0
	for s in secciones:
		if ant==0:
			pass
		else:
			if s.text.strip() in listaDeSecciones:
				html_aux=fhtml[ant:act]
				temp_soup=BeautifulSoup(html_aux,'html.parser')
				# print(temp_soup.get_text())
				dataWriter.writerow([nombre,noCaracteresEspeciales(temp_soup.get_text())])
		ant,act=act,fhtml.find(str(s))

In [ ]:
#archivo utilidades.py

from unicodedata import normalize
import re
def noCaracteresEspeciales(texto):
	# -> NFD y eliminar diacríticos
	texto = re.sub(
        r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
        normalize( "NFD", texto), 0, re.I
    )

	# -> NFC
	texto = normalize( 'NFC', texto)
	texto=re.sub('\W',' ',texto)
	# print(normalize('NFKC',texto))
	return texto.lower()

Hasta este punto tenemos un conjunto de textos sin caracteres especiales ni simbolos de puntuacion, ahora necesitamos convertir esos caracteres en simbolos que la computadora pueda ubicar en una espacio vectorial, para eso voy a utilizar un corpus ya creado de palabras relacionadas con vectores, este corpus es http://crscardellino.github.io/SBWCE/, yo tome los vectores de otro repositorio que tomo estas palabras y entreno una capa de embedding, este es el repositorio https://github.com/uchile-nlp/spanish-word-embeddings, aqui explica bien como usarlo, pero tambien tuve que buscar unos ejemplos de como usar gensim en la siguiente pagina https://radimrehurek.com/gensim/models/fasttext.html. Todo esto lo conjunte en un solo programa que comparará la similaridad entre textos para precedir cual es la enfermedad, el codigo es el siguiente: 

In [ ]:
from gensim.models.keyedvectors import KeyedVectors
import sys
import csv
from utilidades import *
import numpy as np
from scipy import spatial


def printD(di):
	s = [(k, di[k]) for k in sorted(di, key=di.get, reverse=True)]
	print("\n".join(map(lambda x:" -> ".join(map(str,x)),s)))


def vectorPromedioDeTexto(words, model, num_features):
	# Funcion modificada de https://datascience.stackexchange.com/questions/23969/sentence-similarity-prediction
	featureVec = np.zeros((num_features,), dtype="float32")
	nwords=0
	for word in words:
		try:
			nwords = nwords+1
			featureVec = np.add(featureVec, model[word])
		except:
			pass #ignoramos la palabra que conocemos
	if nwords>0:
		featureVec = np.divide(featureVec, nwords)
	return featureVec

def predecirSimilaridad(vector1,vector2):
	return 1 - spatial.distance.cosine(vector1,vector2)


wordvectors_file_vec = 'fasttext-sbwc.3.6.e20.vec.gz'
cantidad = 100000
num_features=300
wordvectors = KeyedVectors.load_word2vec_format(wordvectors_file_vec, limit=cantidad)


corpus={}
with open('datos.csv',"r",newline='') as csvfile:
	spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
	for row in spamreader:
		corpus[row[0]]=vectorPromedioDeTexto(textoALista(row[1]),wordvectors,num_features)
		# print(corpus[row[0]])


with open(sys.argv[1],"r") as f:
	sintomas=vectorPromedioDeTexto(textoALista(noCaracteresEspeciales(f.read())),wordvectors,num_features)

vectorDeSimilaridades={}
for nombre,enfermedad in corpus.items():
	vectorDeSimilaridades[nombre]=predecirSimilaridad(enfermedad,sintomas)

vectorDeSimilaridades={}
for nombre,enfermedad in corpus.items():
	vectorDeSimilaridades[nombre]=predecirSimilaridad(enfermedad,sintomas)

printD(vectorDeSimilaridades)

Para esto tuve que aumentar la biblioteca utilidades con el siguiente codigo:

In [ ]:
def textoALista(texto):
	listaaux=texto.split()
	listaaux.sort()
	lista=[]
	for i in listaaux:
		if i not in lista:
			lista.append(i)
	return lista

Tomo la siguiente entrada:

Tengo dolor de espalda baja por las noches, mejora con el ejercicio y antibioticos, tuve una uveitis en mi ojo izquierdo, ademas de dolor en el cuello y articulaciones de las manos.

Y dio como resultado lo siguiente:

artrosis.html -> 0.890708863735199
ciatica.html -> 0.8889318704605103
bursitis.html -> 0.8863767385482788
artritis-reumatoide.html -> 0.8811296224594116
varices.html -> 0.8773744702339172
pie-atleta.html -> 0.877256453037262
espondilitis-anquilosante.html -> 0.8763821721076965
pies-cavos.html -> 0.8754149675369263
angina-pecho.html -> 0.8753175735473633
juanetes.html -> 0.8747467994689941
torticolis.html -> 0.873279869556427
hiperhidrosis.html -> 0.8719350695610046
espolon.html -> 0.8711838126182556
unas-encarnadas.html -> 0.8708946704864502
callos.html -> 0.8693822026252747
sindrome-del-tunel-carpiano.html -> 0.8691951036453247
migranas.html -> 0.8688270449638367
hipermetropia.html -> 0.8662351369857788
acidez-estomago.html -> 0.8660438060760498
hemorroides.html -> 0.8655730485916138
lumbalgia.html -> 0.8654947876930237
hipotension.html -> 0.8653978705406189
hidradenitis-supurativa.html -> 0.8652818202972412
sinus-pilonidal.html -> 0.8648345470428467
cancer-cabeza-cuello.html -> 0.8626042008399963
sinusitis.html -> 0.8611723184585571
tendinitis.html -> 0.8607843518257141
hernia-inguinal.html -> 0.8607505559921265
melasma.html -> 0.860697329044342
trastornos-ritmo-circadiano.html -> 0.8603765964508057
fascitis-plantar.html -> 0.8595451712608337
dismorfofobia.html -> 0.8579419851303101
pies-zambos.html -> 0.8577716946601868
enfermedad-de-Crohn.html -> 0.857182502746582
bartolinitis.html -> 0.8566218018531799
rectocele.html -> 0.8561906218528748
parkinson.html -> 0.8558446764945984
glaucoma.html -> 0.8554088473320007
enfermedad-tromboembolica-venosa.html -> 0.8552738428115845
urticaria.html -> 0.8550090193748474
hirsutismo.html -> 0.8549813628196716
incontinencia-urinaria.html -> 0.8548083901405334
rinitis.html -> 0.8529385924339294
bruxismo.html -> 0.852780818939209
apnea.html -> 0.8524450063705444
vegetaciones.html -> 0.8521919846534729
insuficiencia-cardiaca.html -> 0.8519532084465027
trastorno-conducta-fase-rem.html -> 0.8517641425132751
esclerosis-multiple.html -> 0.8517006039619446
epoc.html -> 0.8511666655540466
sindrome-de-reiter.html -> 0.8511610627174377
cataratas.html -> 0.8508043885231018
miopia.html -> 0.8503623008728027
ataxia.html -> 0.850346565246582
hernia-discal.html -> 0.8503432273864746
herpes-zoster.html -> 0.8501757979393005
enfermedad-de-whipple.html -> 0.8500857353210449
estrabismo.html -> 0.8499641418457031
cancer-de-faringe.html -> 0.8498618006706238
faringitis.html -> 0.8494998812675476
estrenimiento.html -> 0.8491919040679932
dermatitis-atopica.html -> 0.8489593267440796
faringoamigdalitis.html -> 0.8487638831138611
arritmias.html -> 0.8484375476837158
dispepsia.html -> 0.8484349250793457
sindrome-guillain-barre.html -> 0.8480349779129028
vertigo.html -> 0.8479865193367004
miocardiopatia.html -> 0.8472995162010193
dismenorrea.html -> 0.8472038507461548
bronquiectasias.html -> 0.8466354608535767
cancer-de-colon.html -> 0.8464849591255188
vitiligo.html -> 0.8462146520614624
hipertiroidismo.html -> 0.8459913730621338
trastorno-menstruacion.html -> 0.8453294038772583
desprendimiento-retina.html -> 0.8448761701583862
apendicitis.html -> 0.8447700142860413
colico-lactante.html -> 0.8443443775177002
hipertension-arterial.html -> 0.84421706199646
epilepsia.html -> 0.8442100286483765
cancer-vejiga.html -> 0.8439739942550659
aneurisma.html -> 0.8438948392868042
ictus.html -> 0.8435802459716797
gota.html -> 0.8433400392532349
insomnio.html -> 0.8429895639419556
impetigo.html -> 0.8427833318710327
cancer-prostata.html -> 0.8420568108558655
cancer-de-pancreas.html -> 0.8417086601257324
sonambulismo.html -> 0.8416820764541626
infarto-miocardio.html -> 0.8416303396224976
sindrome-de-marfan.html -> 0.8413761854171753
ela.html -> 0.841342568397522
ojo-seco.html -> 0.841338574886322
colico-nefritico.html -> 0.8410360813140869
cefaleas.html -> 0.8408526182174683
vigorexia.html -> 0.8408465385437012
cervicitis.html -> 0.840455174446106
retinopatia-diabetica.html -> 0.840327799320221
acne.html -> 0.8398477435112
ojo-vago.html -> 0.8396686315536499
corte-digestion.html -> 0.8396542072296143
endocarditis.html -> 0.8395729064941406
vulvitis.html -> 0.8394715189933777
derrame-pleural.html -> 0.8393393754959106
legionella.html -> 0.8393082618713379
asma.html -> 0.8389927744865417
amigdalitis.html -> 0.838949978351593
lepra.html -> 0.8389075994491577
golpe-de-calor.html -> 0.8387756943702698
chikungunya.html -> 0.8386859893798828
aterosclerosis.html -> 0.8386574983596802
sarcoma.html -> 0.8383640050888062
disfuncion-sexual-femenina.html -> 0.8383496999740601
escoliosis.html -> 0.8383411169052124
anorexia.html -> 0.8382319211959839
sindrome-de-fatiga-cronica.html -> 0.8376734256744385
gripe.html -> 0.8375241756439209
psoriasis.html -> 0.8369935750961304
muerte-subita-cardiaca.html -> 0.8369080424308777
neumotorax.html -> 0.8365309834480286
coronavirus.html -> 0.836329996585846
agorafobia.html -> 0.8360276222229004
mononucleosis.html -> 0.8358700275421143
vasculitis.html -> 0.8358007669448853
aftas-bucales.html -> 0.8356896042823792
tos-ferina.html -> 0.835590124130249
enfermedad-de-wilson.html -> 0.8354955911636353
bronquitis.html -> 0.8351003527641296
dislexia.html -> 0.8350762724876404
esofagitis-eosinofilica.html -> 0.8346531391143799
pies-planos.html -> 0.8345018625259399
osteoporosis.html -> 0.8342934250831604
osteosarcoma.html -> 0.8342751264572144
cancer-cuello-utero.html -> 0.8336853384971619
diabetes.html -> 0.8333421945571899
difteria.html -> 0.8326854705810547
escarlatina.html -> 0.8325850367546082
cancer-de-tiroides.html -> 0.8325832486152649
pericarditis.html -> 0.8323855996131897
bullying.html -> 0.8318314552307129
tuberculosis.html -> 0.8317189812660217
enfermedad-de-paget.html -> 0.8316428065299988
alzheimer.html -> 0.8314484357833862
fibrosis-pulmonar.html -> 0.831217348575592
pancreatitis.html -> 0.8312006592750549
ludopatia.html -> 0.831157922744751
hemorragias-ginecologicas.html -> 0.8309376239776611
cancer-ovarios.html -> 0.8307660222053528
sindrome-de-burnout.html -> 0.8307101726531982
depresion.html -> 0.8303148746490479
retinoblastoma.html -> 0.8299978971481323
hiperplasia-benigna-de-prostata.html -> 0.8299832940101624
diarrea.html -> 0.8298776745796204
fiebre-tifoidea.html -> 0.8296930193901062
enfisema.html -> 0.8295069932937622
presbicia.html -> 0.8293984532356262
hemocromatosis.html -> 0.8291944861412048
sindrome-postvacacional.html -> 0.8289538621902466
tumores-cerebrales.html -> 0.8288031220436096
candidiasis.html -> 0.8285830616950989
melanoma-metastasico.html -> 0.8282124996185303
herpes-labial.html -> 0.8282058238983154
litiasis-renal.html -> 0.8280899524688721
hipotiroidismo.html -> 0.8279060125350952
siringomielia.html -> 0.8278234601020813
fobia-social.html -> 0.8277742266654968
factores-de-riesgo-cardiovascular.html -> 0.8274507522583008
linfoma.html -> 0.8274443745613098
sindrome-de-sjogren.html -> 0.8273062705993652
sarampion.html -> 0.8272988200187683
fibrosis-quistica.html -> 0.8271623849868774
colitis-ulcerosa.html -> 0.8265323638916016
prostatitis.html -> 0.8264909386634827
sindrome-de-down.html -> 0.8264230489730835
cancer-testiculo.html -> 0.8263432383537292
sifilis.html -> 0.8262447118759155
enuresis.html -> 0.8261653184890747
fibrosis-pulmonar-idiopatica.html -> 0.8255593180656433
melanoma.html -> 0.82553631067276
malaria.html -> 0.8253500461578369
aerofagia.html -> 0.8251122236251831
hemofilia.html -> 0.8250989317893982
ansiedad.html -> 0.8249941468238831
colelitiasis.html -> 0.8249314427375793
sindrome-de-gilbert.html -> 0.824849009513855
embolia-pulmonar.html -> 0.8247255682945251
sarcoidosis.html -> 0.8244109153747559
trastorno-obsesivo-compulsivo.html -> 0.8241225481033325
bulimia.html -> 0.8240898847579956
estres.html -> 0.8240541815757751
amenorrea.html -> 0.8240024447441101
dermatitis-del-panal.html -> 0.8238811492919922
demencia-senil.html -> 0.8237936496734619
trastorno-deficit-atencion-hiperactividad.html -> 0.8233283758163452
fibromialgia.html -> 0.8231877684593201
lipotimia.html -> 0.8230554461479187
sindrome-de-wolfram.html -> 0.8227017521858215
lupus.html -> 0.8223026990890503
vaginitis.html -> 0.8220986723899841
hepatitis-c.html -> 0.8220308423042297
neumonia.html -> 0.821611762046814
sindrome-de-patau.html -> 0.8211525082588196
ebola.html -> 0.8209527730941772
cancer-uretra.html -> 0.8207820057868958
alopecia.html -> 0.8207060098648071
anemia.html -> 0.8205870389938354
cancer-de-laringe.html -> 0.8202832937240601
sindrome-hiperestimulacion-ovarica.html -> 0.820146381855011
otitis.html -> 0.8197153210639954
gases-flatulencia.html -> 0.8191633820533752
sindrome-de-turner.html -> 0.8190908432006836
cancer-pulmon.html -> 0.8190857172012329
sepsis.html -> 0.8189959526062012
impotencia-disfuncion-erectil.html -> 0.8186733722686768
orquitis.html -> 0.8186292052268982
hepatitis-a.html -> 0.8176063895225525
obesidad.html -> 0.8171986937522888
fiebre-amarilla.html -> 0.8170220255851746
arterioesclerosis.html -> 0.8168722987174988
endometriosis.html -> 0.816849410533905
esofago-de-barrett.html -> 0.8167792558670044
ulcera-gastroduodenal.html -> 0.8165062069892883
infeccion-urinaria.html -> 0.8161539435386658
colon-irritable.html -> 0.8158481121063232
paperas.html -> 0.8156622648239136
gonorrea.html -> 0.8156198263168335
cancer-piel.html -> 0.8154959678649902
conjuntivitis.html -> 0.8153886795043945
rosacea.html -> 0.8150522112846375
miomas-uterinos.html -> 0.8145495653152466
alergia-al-polen.html -> 0.8144137263298035
leucemia.html -> 0.814260721206665
cancer-de-estomago.html -> 0.8138683438301086
meningitis.html -> 0.8137497901916504
hongos.html -> 0.8137203454971313
celiaquia.html -> 0.8136854767799377
cancer-de-las-vias-biliares.html -> 0.8131875991821289
esofagitis.html -> 0.8130001425743103
rabia.html -> 0.8125672936439514
sindrome-de-rett.html -> 0.8124037384986877
esquizofrenia.html -> 0.8116720914840698
cardiopatia-congenita.html -> 0.8105366826057434
alergias-alimentarias.html -> 0.8100298643112183
mobbing.html -> 0.8094996213912964
cancer-mama.html -> 0.8083365559577942
fiebre-del-heno.html -> 0.808133602142334
adenoma-hipofisiario.html -> 0.807842493057251
sindrome-diogenes.html -> 0.8075513243675232
autismo.html -> 0.8073760867118835
rubeola.html -> 0.8073654174804688
orzuelo.html -> 0.8058894276618958
mieloma-multiple.html -> 0.8058028817176819
enfermedad-del-sueno.html -> 0.8057993650436401
encefalopatia-hepatica.html -> 0.8056855201721191
silicosis.html -> 0.8055431246757507
esclerodermia.html -> 0.805502712726593
hepatitis-b.html -> 0.8039403557777405
botulismo.html -> 0.8038089275360107
hipercolesterolemia.html -> 0.8038040995597839
asbestosis.html -> 0.802758514881134
fiebres-hemorragicas.html -> 0.8023279309272766
cancer-de-higado.html -> 0.8002320528030396
cirrosis.html -> 0.7997555136680603
encefalitis.html -> 0.7996863722801208
astigmatismo.html -> 0.799540638923645
molusco-contagioso.html -> 0.7990315556526184
rotavirus.html -> 0.7986818552017212
narcolepsia.html -> 0.7975443005561829
cancer-de-rinon.html -> 0.7974680662155151
varicela.html -> 0.795936644077301
toxoplasmosis.html -> 0.7942932844161987
colera.html -> 0.7934789657592773
tricomoniasis.html -> 0.7932755947113037
gastroenteritis.html -> 0.7923362851142883
sindrome-de-williams.html.html -> 0.7900022268295288
ortorexia.html -> 0.7894489765167236
eccema.html -> 0.7892985939979553
edema-pulmonar.html -> 0.7892346978187561
esterilidad-infertilidad.html -> 0.7878594994544983
fenilcetonuria.html -> 0.7855566143989563
cancer-de-intestino-delgado.html -> 0.7839733958244324
leishmaniasis.html -> 0.7834895849227905
salmonelosis.html -> 0.783294677734375
sida.html -> 0.7789340019226074
dengue.html -> 0.7769478559494019
alergia.html -> 0.7753075361251831
sindrome-de-estres-postraumatico.html -> 0.774480938911438
infeccion-virus-zika.html -> 0.7732224464416504
gastritis.html -> 0.7717269659042358
poliomielitis.html -> 0.7701467275619507
balanitis.html -> 0.767002284526825
hipoglucemia.html -> 0.7669535875320435
anisakiasis.html -> 0.7614238858222961
creutzfeldt-jakob.html -> 0.7612504959106445
vaginosis-bacteriana.html -> 0.7585615515708923
tetanos.html -> 0.7520037889480591
listeriosis.html -> 0.745862603187561
trastornobipolar.html -> 0.6331714391708374

# Conclusiones
El experimento parece estar cerca de la realidad, la descripcion de los sintomas corresponde a un cuadro de espondilitis anquilosante, la cual se encuentra en el lugar 7 de las enfermedades mas probables, esto puede ser por muchas razones, mi teoria es que necesita mucha mas información sobre la enfermedad, ya que la descripcion de los sintomas era muy general:

Texto correspondiente a la espondilistis anquilosante
que es la espondilitis anquilosante o anquilopoyetica  ea  es una enfermedad reumatica que causa inflamacion de las articulaciones de la columna vertebral y de las sacroiliacas  se suele manifestar con fases de dolor lumbar que pueden afectar a toda la columna y a las articulaciones perifericas  ocasionando dolor en ambas  rigidez vertebral  perdida de movilidad y deformidad articular progresiva  esta patologia puede acompañarse de manifestaciones extraarticulares  como inflamacion en los ojos o en las valvulas del corazon

para manejar mejor estos casos propongo tomar otra pagina web como fuente de datos como la siguiente:
https://www.onmeda.es/enfermedades/espondilitis_anquilosante-sintomas-1552-4.html en la que se explica mas detalladamente los sintomas.
Por otro lado al ubicarse en uno de los primeros lugares y que las primeras enfermedades estan relacionadas con cuadros reumatoides, la inferencia sobre los datos fue correcta, sin embargo me preocupa que todas la enfermedades presentan una similaridad con los sintomas descritos, mas del 50 %, lo que me haria sospechar que necesitamos usar algun metodo mas sofisticado para separar mejor las muestras. Otra cosas que puedo notar es que no tomar palabras muy tecnicas como HLA B27 un factor determinante en el diagnostico de esta enfermedad, probablemente deba entrenar mi propio embedding. 